In [1]:
import os
import sys
import torchvision
import json
from transformers.tokenization_bert import BertTokenizer

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/media/moju/data/work'

In [2]:
sys.path.append(w_dir+'/fgc_support_retri')

In [5]:
import torch
import torch.nn as nn
import torchvision
from transformers.tokenization_bert import BertTokenizer
import config
from fgc_preprocess import FgcSerDataset, BertIdx
from torch.utils.data import DataLoader
from sup_model import BertSupSentClassification
from transformers import BertModel
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm_notebook as tqdm
from fgc_support_retri.fgc_preprocess import HotpotDataset

In [6]:
config.HOTPOT_DEV

PosixPath('/media/moju/data/work/fgc_support_retri/data/hotpot_dataset/FGC_hotpot_dev_distractor_v1(cn_refn).json')

In [5]:
def get_sup(item):
    item['SENTS'] = item['DTEXT'].split('\n')[:-1]
    sup_evidence = [0] * len(item['SENTS'])
    
    for shi in item['SHINT']:
        for s_i, s in enumerate(item['SENTS']):
            if shi['text'] == s:
                sup_evidence[s_i] = 1
    item['SUP_EVIDENCE'] = sup_evidence

In [6]:
with open(config.HOTPOT_DEV) as f:
    items = json.load(f)
print("{} questions".format(len(items)))

for item in items:
    assert len(item['QUESTIONS']) == 1
    for k, v in item['QUESTIONS'][0].items():
        item[k] = v
    del item['QUESTIONS']
    get_sup(item)
    assert 1 in item['SUP_EVIDENCE']
    del item['SHINT']
    del item ['ASPAN']

7405 questions


In [7]:
items[0]

{'DID': '5a8b57f25542995d1e6f1371',
 'DTEXT': '斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。\n他住在加州洛杉矶。\n他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》\n爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。\n',
 'QID': '1',
 'QTEXT': '斯科特·德里克森和埃德·伍德是同一国籍的吗？',
 'QTYPE': '进阶题',
 'AMODE': 'Comparing-Members',
 'ANSWER': [{'ATEXT': '对', 'ATOKEN': {'text': '对', 'start': 0}},
  {'ATEXT': 'yes', 'ATOKEN': {'text': 'yes', 'start': 0}}],
 'SENTS': ['斯科特·德里克森（1966年7月16日出生）是美国导演、编剧和制片人。',
  '他住在加州洛杉矶。',
  '他最出名的是执导恐怖电影，如《险恶》、《艾米丽·罗斯的驱魔》、《让我们远离邪恶》，以及2016年惊奇电影宇宙版的《奇怪医生》',
  '爱德华·戴维斯·伍德（1924年10月10日至1978年12月10日）是美国电影制片人、演员、作家、制片人和导演。'],
 'SUP_EVIDENCE': [1, 0, 0, 1]}

In [8]:
bert_model_name = 'bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
dataset = HotpotDataset(items, transform=BertIdx(tokenizer))

In [9]:
dataset[0].keys()

dict_keys(['DID', 'SID', 'QTEXT', 'sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])

In [10]:
len(dataset[0]['input_ids'])

56

In [11]:
len(dataset[1]['input_ids'])

34

In [12]:
len(dataset)

47988

In [13]:
dataset

In [15]:
# dataloader_train = DataLoader(dataset, batch_size=32, collate_fn=bert_collate)
# for batch in dataloader_train:
#     print(batch)
#     import pdb; pdb.set_trace()